# Project 4: West Nile Virus in the City of Chicago

Michael Schillawski, 9 March 2018

Data Science Immersive, General Assembly

## Setup

### Import

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import datetime
import pandas_profiling as pdp

% matplotlib inline

### Gather Data

In [ ]:
#os.chdir('..')

In [3]:
path = os.getcwd()
path = path + '/assets/input'
os.chdir(path)

In [4]:
ls

mapdata_copyright_openstreetmap_contributors.rds
mapdata_copyright_openstreetmap_contributors.txt
noaa_weather_qclcd_documentation.pdf
sampleSubmission.csv
spray.csv
test.csv
train.csv
weather.csv


In [5]:
train = pd.read_csv('train.csv')
spray = pd.read_csv('spray.csv')
weather = pd.read_csv('weather.csv')
#test = pd.read_csv('test.csv')

## Data Cleaning

### Formatting & DeDuping

#### Inspecting

In [6]:
def DataInspect(dataframe):
    '''Original function (previously called eda) created by Ritika Bhasker
       Good first step when starting any project. Provides overview of
       dataset including missing values, duplicates and types.
       Takes a Pandas dataframe as the argument.
       Modified by Michael Schillawski based on his preferences.'''
    print("Dataframe Shape:", dataframe.shape,"\n")
    print("Duplicate Rows:", dataframe.duplicated().sum(),"\n") #Added this
    print("Dataframe Types \n\n", dataframe.dtypes,"\n")    
    print("Missing Values \n\n", dataframe.isnull().sum(),"\n")
    print("Dataframe Describe \n\n", dataframe.describe(include='all'),"\n")


    print('Unique Values by Variable')
    for item in dataframe:
        print(item,':',dataframe[item].nunique())

In [11]:
DataInspect(train)

Dataframe Shape: (10506, 12) 

Duplicate Rows: 813 

Dataframe Types 

 Date                      datetime64[ns]
Address                           object
Species                           object
Block                              int64
Street                            object
Trap                              object
AddressNumberAndStreet            object
Latitude                         float64
Longitude                        float64
AddressAccuracy                    int64
NumMosquitos                       int64
WnvPresent                         int64
dtype: object 

Missing Values 

 Date                      0
Address                   0
Species                   0
Block                     0
Street                    0
Trap                      0
AddressNumberAndStreet    0
Latitude                  0
Longitude                 0
AddressAccuracy           0
NumMosquitos              0
WnvPresent                0
dtype: int64 

Dataframe Describe 

                        Date  

In [12]:
DataInspect(spray)

Dataframe Shape: (14835, 4) 

Duplicate Rows: 541 

Dataframe Types 

 Date         datetime64[ns]
Time                 object
Latitude            float64
Longitude           float64
dtype: object 

Missing Values 

 Date           0
Time         584
Latitude       0
Longitude      0
dtype: int64 

Dataframe Describe 

                        Date        Time      Latitude     Longitude
count                 14835       14251  14835.000000  14835.000000
unique                   10        8583           NaN           NaN
top     2013-08-15 00:00:00  7:44:32 PM           NaN           NaN
freq                   2668         541           NaN           NaN
first   2011-08-29 00:00:00         NaN           NaN           NaN
last    2013-09-05 00:00:00         NaN           NaN           NaN
mean                    NaN         NaN     41.904828    -87.736690
std                     NaN         NaN      0.104381      0.067292
min                     NaN         NaN     41.713925    -88.09646

In [13]:
DataInspect(weather)

Dataframe Shape: (2944, 22) 

Duplicate Rows: 0 

Dataframe Types 

 Station                 int64
Date           datetime64[ns]
Tmax                    int64
Tmin                    int64
Tavg                   object
Depart                 object
DewPoint                int64
WetBulb                object
Heat                   object
Cool                   object
Sunrise                object
Sunset                 object
CodeSum                object
Depth                  object
Water1                 object
SnowFall               object
PrecipTotal            object
StnPressure            object
SeaLevel               object
ResultSpeed           float64
ResultDir               int64
AvgSpeed               object
dtype: object 

Missing Values 

 Station        0
Date           0
Tmax           0
Tmin           0
Tavg           0
Depart         0
DewPoint       0
WetBulb        0
Heat           0
Cool           0
Sunrise        0
Sunset         0
CodeSum        0
Depth          0

In [ ]:
pdp.ProfileReport(train)

In [ ]:
pdp.ProfileReport(spray)

In [ ]:
pdp.ProfileReport(weather)

#### Fix dates

In [7]:
train['Date'] = pd.to_datetime(train['Date'])
spray['Date'] = pd.to_datetime(spray['Date'])
weather['Date'] = pd.to_datetime(weather['Date'])
#test['Date'] = pd.to_datetime(test['Date'])

In [8]:
spray['Date'].describe()

count                   14835
unique                     10
top       2013-08-15 00:00:00
freq                     2668
first     2011-08-29 00:00:00
last      2013-09-05 00:00:00
Name: Date, dtype: object

In [9]:
train['Date'].describe()

count                   10506
unique                     95
top       2007-08-01 00:00:00
freq                      551
first     2007-05-29 00:00:00
last      2013-09-26 00:00:00
Name: Date, dtype: object

In [10]:
weather['Date'].describe()

count                    2944
unique                   1472
top       2014-08-27 00:00:00
freq                        2
first     2007-05-01 00:00:00
last      2014-10-31 00:00:00
Name: Date, dtype: object

#### Drop Duplicates

In [19]:
# inspect duplicates in spray
train[train.duplicated(keep=False)][:5]

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent


In [18]:
train.drop(train[train.duplicated(keep='first')].index,axis=0,inplace=True)

In [20]:
# inspect duplicates in spray
spray[spray.duplicated(keep=False)][:5]

,Date,Time,Latitude,Longitude
484,2011-09-07,7:43:40 PM,41.983917,-87.793088
485,2011-09-07,7:43:40 PM,41.983917,-87.793088
489,2011-09-07,7:44:32 PM,41.986460,-87.794225
490,2011-09-07,7:44:32 PM,41.986460,-87.794225
491,2011-09-07,7:44:32 PM,41.986460,-87.794225


In [21]:
spray.drop(spray[spray.duplicated(keep='first')].index,axis=0,inplace=True)

In [22]:
print(len(train))
print(len(spray))
print(len(weather))

9693
14294
2944


#### Dummy species

In [23]:
# get dummies for mosquito species

spc_d = pd.get_dummies(train, columns=['Species'])
traincol = train.columns.drop('Species')
train = pd.concat([train[traincol],spc_d])

#### Fix weather data

In [24]:
# Changes string T to 0.005, M to 0.0 (11 observations of 2944), and everything else into a float
def rainy_day(column):
    weather[column] = weather[column].str.replace('T','0.005')
    weather[column] = weather[column].str.replace('M','0.0')
    weather[column] = weather[column].astype(float)

In [25]:
rainy_day('Tavg')
rainy_day('PrecipTotal')

In [26]:
#All Tavg values that were missing (changed to 0 with rainy_day function) are given values from min/max
weather['Tavg'][weather.Tavg==0] = (weather['Tmin'] + weather['Tmax']) / 2
weather['Tavg'].value_counts()

/Users/mjschillawski/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


73.0    138
70.0    117
77.0    117
75.0    110
71.0    109
74.0    107
72.0    104
78.0    103
69.0    103
76.0    100
68.0     99
79.0     98
66.0     94
67.0     89
61.0     88
64.0     86
65.0     84
80.0     84
63.0     81
62.0     67
57.0     67
60.0     61
50.0     57
81.0     55
53.0     49
54.0     49
58.0     49
55.0     48
82.0     48
56.0     46
       ... 
46.0     24
47.0     24
84.0     21
44.0     19
48.0     17
85.0     16
86.0     16
42.0     15
43.0     12
87.0      9
41.0      7
40.0      5
88.0      4
91.0      4
39.0      4
89.0      4
38.0      2
36.0      2
90.0      2
37.0      2
71.5      1
92.0      1
94.0      1
64.5      1
93.0      1
63.5      1
56.5      1
85.5      1
67.5      1
68.5      1
Name: Tavg, Length: 66, dtype: int64

## EDA

### Join Weather Data to Trap Data

In [27]:
print(len(train),len(weather))

19386 2944


In [28]:
weather.columns

Index(['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint',
       'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'Depth',
       'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel',
       'ResultSpeed', 'ResultDir', 'AvgSpeed'],
      dtype='object')

In [29]:
station1 = weather[weather['Station']==1].copy()
station2 = weather[weather['Station']==2].copy()


#Station 1: CHICAGO O'HARE INTERNATIONAL AIRPORT Lat: 41.995 Lon: -87.933 Elev: 662 ft. above sea level
station1['Latitude'] = 41.995
station1['Longitude'] = -87.9336

#Station 2: CHICAGO MIDWAY INTL ARPT Lat: 41.786 Lon: -87.752 Elev: 612 ft. above sea level
station2['Latitude'] = 41.78611
station2['Longitude'] = -87.75222


In [30]:
stations = pd.merge(station1,station2,on='Date',suffixes=('_s1','_s2'))
traps_weather = pd.merge(train,stations,on='Date')

In [31]:
DataInspect(traps_weather)

Dataframe Shape: (19386, 64) 

Duplicate Rows: 699 

Dataframe Types 

 Address                                   object
AddressAccuracy                            int64
AddressNumberAndStreet                    object
Block                                      int64
Date                              datetime64[ns]
Latitude                                 float64
Longitude                                float64
NumMosquitos                               int64
Species_CULEX ERRATICUS                  float64
Species_CULEX PIPIENS                    float64
Species_CULEX PIPIENS/RESTUANS           float64
Species_CULEX RESTUANS                   float64
Species_CULEX SALINARIUS                 float64
Species_CULEX TARSALIS                   float64
Species_CULEX TERRITANS                  float64
Street                                    object
Trap                                      object
WnvPresent                                 int64
Station_s1                                 int

### Calculate point estimates of weather data at trap location

In [36]:
# calculate distance of traps to weather stations
dist_1 = np.sqrt(((traps_weather['Latitude'] - traps_weather['Latitude_s1'])**2 + 
 (traps_weather['Longitude'] - traps_weather['Longitude_s1'])**2))

dist_2 = np.sqrt(((traps_weather['Latitude'] - traps_weather['Latitude_s2'])**2 + 
 (traps_weather['Longitude'] - traps_weather['Longitude_s2'])**2))

# calculate distance weights to each trap
# to weight the weather data by proximity
total_dist = dist_1 + dist_2
traps_weather['weight_1'] = dist_1 / total_dist
traps_weather['weight_2'] = dist_2 / total_dist

In [37]:
# Apply distance weights to weather data
# Inverse weight because the closer station should have the heavier weight

station1_list = [col for col in traps_weather.columns 
                if '_s1' in col and col not in ('Station_s1','Latitude_s1','Longitude_s1','CodeSum_s1')]
station2_list = [col for col in traps_weather.columns 
                 if '_s2' in col and col not in ('Station_s2','Latitude_s2','Longitude_s2','CodeSum_s2')]

for col in station1_list:
    traps_weather[col] = traps_weather['weight_2'] * traps_weather[col]
for col in station2_list:
    traps_weather[col] = traps_weather['weight_1'] * traps_weather[col]

TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
for col in [col for col in traps_weather.columns 
            if 's1' in col and col not in ('Station_s1','Latitude_s1','Longitude_s1')]:
    name = col.replace('_s1','')
    traps_weather[name] = traps_weather[col] + traps_weather[name+'_s2']

### Spraying Data

In [ ]:
len(spray)

In [ ]:
spray.head(3)

In [ ]:
# the idea here is that the targeted intervention has an effect that decays in two dimensions, time and distance
# the reference location that we care about is the trap
# so we evaluate spraying by how far the trap is from the where the spraying occurs AND
# we evaluate how long before the trap observation did the spraying occur
# so we calculate the deltas for every spraying against every trap
# if spraying occurred after the observation, we zero out these observations

# should we cross-multiply the distance and time? spraying that is 
# close in both time and distance should be privileged

distance = []
time = []

for i in traps_weather.index:
    temp_lat = traps_weather.at[i,'Latitude']
    temp_long = traps_weather.at[i,'Longitude']

    # calculate distance from traps to spray locations
    dist = np.sqrt((spray['Latitude'] - temp_lat)**2 + (spray['Longitude'] - temp_long)**2)
    distance.append(dist)

    # calculate time since spray
    time_since_spray = traps_weather.at[i,'Date'] - spray['Date']
    time.append(time_since_spray)

distance = pd.DataFrame(distance)
time = pd.DataFrame(time)

time.reset_index(inplace=True)
time.drop('index',axis=1,inplace=True)

In [ ]:
backup = time.copy()

In [ ]:
distance.head()

In [ ]:
time.head()

In [ ]:
from datetime import timedelta 

for col in time.columns:
    time[col] = time[col].dt.total_seconds()
    #time[col] = time[col].apply(lambda x: (((x/60)/60)/24))

In [ ]:
time.head()

In [ ]:
data = pd.merge(distance,time,how='inner',left_index=True,right_index=True,suffixes=('_d','_t'))
data.shape

### Outcome variable(s)

In [ ]:
# trim dataset
# drop string geographic variables, info encoded in lat/long
# drop _s1/_s2 weather variables --> stored after weighted average across distances

In [ ]:
from sklearn.model_selection import train_test_split

X = data.drop('WnvPresent',axis=1)
Y = data[['WnvPresent']]

X_train,X_test,y_train,y_test = train_test_split()

## Modeling

We need to predict when, where, and among which species West Nile Virus will occur
In which traps will we observe West Nile Virus?

How to define outcome variable?

Outcome: WNV 1/0

Variable selection:
- weather
- spray
- species
- location-based

In [ ]:
### Logistic Regression

In [ ]:
### KNN

In [ ]:
### Random Forest

In [ ]:
### ExtraTrees

In [ ]:
### Adaboost

In [ ]:
### Gradient Boosting

predict when, where, which we would expect to see West Nile Virus without intervention
without spraying in model, where do the predictions deviate?